In [1]:
from selenium import webdriver
from lxml import etree
import time
import itertools
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import re
import math

In [2]:
driver = webdriver.Chrome(executable_path=r'D:/chromedriver_win32/chromedriver')

In [3]:
url = 'https://www.booking.com'
driver.get(url)
input_box=driver.find_element_by_xpath('//input[@class="c-autocomplete__input sb-searchbox__input sb-destination__input"]')
input_box.clear()
input_box.send_keys('Nashville')
time.sleep(2)
date_box=driver.find_element_by_xpath('//div[@class="xp__input-group xp__date-time"]').click()
check_in_date=driver.find_element_by_xpath('//td[@data-date="2021-06-29"]').click()
time.sleep(1)
check_out_date=driver.find_element_by_xpath('//td[@data-date="2021-06-30"]').click()        
time.sleep(5)

search_button=driver.find_element_by_xpath('//div[@class="sb-searchbox-submit-col -submit-button "]').click()

In [4]:
html = etree.HTML(driver.page_source)
num_hotel = html.xpath('//div[@class="sr_header "]/h1/text()')
temp = re.findall(r'\d+', num_hotel[0]) 
num_hotel = int(list(map(int, temp))[0])

page = num_hotel/25
page=math.ceil(page)+1

In [5]:
print("Total Pages: "+str(page))

Total Pages: 9


In [6]:
hotel_names = []
review_scores = []
nightly_prices = []
hotel_links = []
review_nums = []
hotel_latlngs=[]

In [7]:
for i in range (0,page)[:1]:
    try:
        url = driver.current_url[:-1]+str(25*i)
        driver.get(url)
        time.sleep(2)

        y = 800
        for timer in range(0,12):
            driver.execute_script("window.scrollTo(0, "+str(y)+")")
            y += 1000  
            time.sleep(1)

        html = etree.HTML(driver.page_source) 

        hotel_name = html.xpath('//a[@class="js-sr-hotel-link hotel_name_link url"]/span[1]/text()')
        review_score = html.xpath('//div[@class="bui-review-score__badge"]/text()')
        nightly_price = html.xpath('//div[@class="prco-inline-block-maker-helper"]/span[1]/text()')
        hotel_link = html.xpath('//a[@class="js-sr-hotel-link hotel_name_link url"]/@href')
        review_num = html.xpath('//div[@class="bui-review-score__text"]/text()')
        hotel_latlng = html.xpath('//a[@class="bui-link"]/@data-coords')

        for name in hotel_name:
                    hotel_names.append(name)
        for score in review_score:
                    review_scores.append(score)
        for price in nightly_price:
                    nightly_prices.append(price)
        for link in hotel_link:
                    hotel_links.append(link)        
        for num in review_num:
                    review_nums.append(num)
        for latlng in hotel_latlng:
                    hotel_latlngs.append(latlng)
    except:
        None

In [8]:
hotelreview = pd.DataFrame(
    {'hotel_name': hotel_names,
     'review_score': review_scores,
     'nightly_price': nightly_prices,
     'hotel_link' : hotel_links,
     'review_num':review_nums,
     'hotel_latlng' :hotel_latlngs
    })

In [9]:
hotelreview.to_csv('data/hotelreview_raw.csv',index=False)

In [10]:
hotelreview.head(3)

,hotel_name,review_score,nightly_price,hotel_link,review_num,hotel_latlng
0,"\nHoliday Inn Nashville Vanderbilt, an IHG Hot...",8.5,\nPrice\n$170\n,\n/hotel/us/holiday-inn-nashville-vanderbilt-d...,"1,169 reviews","-86.8113416433334,36.1450616835649"
1,\nFairfield Inn & Suites by Marriott Nashville...,9.0,\nPrice\n$159\n,\n/hotel/us/fairfield-inn-suites-by-marriott-n...,201 reviews,"-86.797119,36.188747"
2,\nComfort Inn Nashville – Opryland Area\n,8.3,\nPrice\n$132\n,\n/hotel/us/days-inn-opryland.html?label=gen17...,343 reviews,"-86.694689,36.222487"
